In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
#os.getenv('OPENAI_API_KEY')

In [4]:
import json
from openai import OpenAI

In [5]:
client = OpenAI()

### Function Calling
connect large language models to external tools

In [6]:
prompt = f'''
please extract the following information from the given text in JSON format.

year
winner_team
runner_up_team

Information:
First cricket worldcup was played in 1975 between West Indies and Australia. This worldcup was won by West Indies.
'''
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    max_tokens=50,
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ]
)
json.loads(response.choices[0].message.content)

{'year': 1975, 'winner_team': 'West Indies', 'runner_up_team': 'Australia'}

#### Same as above but using Function Calling

In [9]:
functions = [
    {
        "name": "extract_details",
        "parameters": {
            "type": "object",
            "properties": {
                "year": {"type": "integer"},
                "winner_team": {"type": "string"},
                "runner_up_team": {"type": "string"},
            }
        }
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    max_tokens=50,
    functions=functions,
    function_call="auto",
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ]
)
response.choices[0].message

# expects a function call named extract_details()

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{}', name='extract_details'), tool_calls=None)

### Inline function call

In [13]:
from datetime import datetime, timedelta

prompt = "when is the next cricket match between pakistan and india?"

def get_next_match(team1: str, team2: str):
    # dummy function to return next match date between two teams. this could be replaced with espncricinfo api to get real data
    return json.dumps({"team1": team1, "team2": team2, "next_match_date": str(datetime.today() + timedelta(days=7))})

functions = [
    {
        "name": "get_next_match",
        "parameters": {
            "type": "object",
            "properties": {
                "team1": {"type": "string"},
                "team2": {"type": "string"},
                "next_match_date": {"type": "string"},
            },
            "required": ["team1", "team2"]
        }
    }
]

response1 = client.chat.completions.create(
    model="gpt-3.5-turbo",
    max_tokens=50,
    functions=functions,
    function_call="auto",
    messages=[
        {"role": "user", "content": prompt}
    ]
)
params = json.loads(response1.choices[0].message.function_call.arguments)
get_next_match_func = eval(response1.choices[0].message.function_call.name)(**params)

response2 = client.chat.completions.create(
    model="gpt-3.5-turbo",
    max_tokens=50,
    functions=functions,
    function_call="auto",
    messages=[
        {"role": "user", "content": prompt},
        {"role": "function", "name": response1.choices[0].message.function_call.name, "content": get_next_match_func}
    ]
)

print(response2.choices[0].message)
print(response2.choices[0].message.content)

ChatCompletionMessage(content='The next cricket match between Pakistan and India is scheduled for July 7, 2024 at 14:57:25.549069 UTC.', role='assistant', function_call=None, tool_calls=None)
The next cricket match between Pakistan and India is scheduled for July 7, 2024 at 14:57:25.549069 UTC.
